<a href="https://colab.research.google.com/github/padfoot194/Pytorch_Learning/blob/main/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Convert MNIST image files into a tensor of 4-Dimension
#(no. of images, height, width, color channel)

transform= transforms.ToTensor()

In [3]:
#Download and Train data
train_data= datasets.MNIST(root='/CNN_data', train=True, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.37MB/s]


In [4]:
#Download and Test data
test_data= datasets.MNIST(root='/CNN_data', train=False, download=True, transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /CNN_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /CNN_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
#Create a small batchsize for images...let's say 10
train_loader= DataLoader(train_data, batch_size=10, shuffle=True)
test_loader= DataLoader(test_data, batch_size=10, shuffle=False)

In [8]:
# Define our CNN Model
# Describe convolutional layer and what it's doing.(2 convolutional layer)

#Conv2d(input_layer,output_layer,kernel_size,stride or step to move)
conv1=nn.Conv2d(1,6,3,1)
conv2=nn.Conv2d(6,16,3,1)

In [9]:
# Grab one MNIST record/an image

for i, (X_train,y_train) in enumerate(train_data):
  break

In [10]:
X_train.shape

torch.Size([1, 28, 28])

In [11]:
# So, we've pulled out one image of 28X28 size. So we've grabbed one image

In [12]:
#Convert to 4D image

x= X_train.view(1,1,28,28)

In [13]:
#Let's perform our first convolution

x=F.relu(conv1(x))

In [14]:
x.shape

torch.Size([1, 6, 26, 26])

In [15]:
# 1 single image, 6 is the filters we've asked for in conv1.
#then comes the image size.it should be 28X28. but it is 26X26
#We didn't do any padding.So,at the edge of the image,some datapoints are
#lost.In real world problem,we should do padding to not lose any datapoint.

In [16]:
#Pass through the pooling layer.
x= F.max_pool2d(x,2,2)  #kernel of 2 and stride of 2.

In [17]:
x.shape

torch.Size([1, 6, 13, 13])

In [18]:
## 1 single image, 6 is the filters we've asked for in conv1.
#then comes the pooling. informations are shrunk in pooling.
#Thus a pooling layer is done.

In [19]:
#Do our second convolutional layer.

x=F.relu(conv2(x))

In [20]:
x.shape

torch.Size([1, 16, 11, 11])

In [21]:
#13X13 reduces to 11X11. Because no padding. Lose 2 pixels around the image.

In [22]:
#Another pooling layer.
x=F.max_pool2d(x,2,2)

In [23]:
x.shape

torch.Size([1, 16, 5, 5])

In [24]:
# Model class

class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1= nn.Conv2d(1,6,3,1)
    self.conv2= nn.Conv2d(6,16,3,1)

    #Fully connected layer
    self.fc1= nn.Linear(5*5*16,120) #Flattening out to 120 neuron.
    self.fc2= nn.Linear(120,84)     #ager output 120 ekhane input.
    self.fc3= nn.Linear(84,10)      #ager output 84 ekhane input.

  def forward(self,X):
    X=F.relu(self,conv1(X))
    X=F.max_pool2d(X,2,2)         # 2X2 kernel and stride 2

    #Second pass
    X=F.relu(self,conv2(X))
    X=F.max_pool2d(X,2,2)         # 2X2 kernel and stride 2

    #Re-view to flatten it out.
    X= X.view(-1, 16*5*5)   #Negative 1 so that we can vary the batch size.

    #Fully connected Layer
    X= F.relu(self.fc1(X))
    X= F.relu(self.fc2(X))
    X= self.fc3(X)

    return F.log_softmax(X, dim=1)

In [25]:
# Create an instance of our model

torch.manual_seed(41)
model= ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [26]:
# Loss function optimizer

criterion= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr=0.001)
#Smaller the lr, the longer it's gonna take to train.